In [ ]:
# The Big Picture
# Compositional screening answers the question:
# "In a given chemical system, what compositions are chemically feasible, and which ones have actually been made?"

# The gap between what's theoretically possible and what's been synthesised
# represents opportunities for new battery materials discovery.

# Let's explore this systematically using the Na–Fe–O system!

# ----------------------------
# 🔧 Import Libraries
# ----------------------------

import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# Materials science libraries
from pymatgen.core import Composition
from mp_api.client import MPRester  # Requires MP API key set as environment variable

# SMACT for chemical screening
import smact
from smact import Element
from smact.screening import smact_validity

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")

# ----------------------------
#  Project-Specific Message
# ----------------------------
print("✓ Libraries imported successfully!")
print(" Ready to begin compositional screening of Na–Fe–O system for battery materials")


✓ Libraries imported successfully!
 Ready to begin compositional screening of Na–Fe–O system for battery materials


In [ ]:
!pip install pymatgen mp-api smact matplotlib seaborn plotly pandas numpy

In [ ]:
# Step 1: Define Our Chemical System
#  Target system: Sodium-Iron-Oxygen for battery materials

CHEMICAL_SYSTEM = ["Na", "Fe", "O"]
MAX_STOICH = 8  # Maximum stoichiometry to consider

# Create SMACT Element objects
from smact import Element

elements = [Element(symbol) for symbol in CHEMICAL_SYSTEM]

# Print system overview
print(f" Target Chemical System: {'-'.join(CHEMICAL_SYSTEM)}")
print(f"  Maximum stoichiometry threshold: {MAX_STOICH}")
print(" Elements loaded with oxidation states:")

for el in elements:
    ox_states = ", ".join([f"{ox:+d}" for ox in el.oxidation_states[:5]])  # Show up to 5
    more = "..." if len(el.oxidation_states) > 5 else ""
    print(f"   {el.symbol}: oxidation states [{ox_states}{more}]")

print("\n Chemical system defined successfully and ready for screening!")


 Target Chemical System: Na-Fe-O
  Maximum stoichiometry threshold: 8
 Elements loaded with oxidation states:
   Na: oxidation states [+1]
   Fe: oxidation states [+1, +2, +3, +4, +5...]
   O: oxidation states [-2, -1]

 Chemical system defined successfully and ready for screening!


In [ ]:
print(" Generating all possible compositions in the Na–Fe–O system...")

# Generate compositions for all possible element combinations
from pymatgen.core import Composition
import itertools

all_compositions = []

# 1. Unary compounds (single elements)
for element in elements:
    comp = Composition({element.symbol: 1})
    all_compositions.append(comp.reduced_composition)

# 2. Binary compounds (two elements)
for el1, el2 in itertools.combinations(elements, 2):
    for stoich1 in range(1, MAX_STOICH + 1):
        for stoich2 in range(1, MAX_STOICH + 1):
            comp = Composition({el1.symbol: stoich1, el2.symbol: stoich2})
            all_compositions.append(comp.reduced_composition)

# 3. Ternary compounds (all three elements)
for stoich_na in range(1, MAX_STOICH + 1):
    for stoich_fe in range(1, MAX_STOICH + 1):
        for stoich_o in range(1, MAX_STOICH + 1):
            comp = Composition({"Na": stoich_na, "Fe": stoich_fe, "O": stoich_o})
            all_compositions.append(comp.reduced_composition)

# Remove duplicates by converting to set
unique_compositions = list(set(all_compositions))

# Print summary
print(f"\n Generated {len(unique_compositions)} unique compositions")

# Show a few examples
print("\n Examples of generated compositions:")
for i, comp in enumerate(unique_compositions[:8]):
    print(f"  {i+1}. {comp}")
if len(unique_compositions) > 8:
    print(f"  ... and {len(unique_compositions) - 8} more")

print("\n This is the 'combinatorial explosion' we discussed earlier!")


 Generating all possible compositions in the Na–Fe–O system...

 Generated 571 unique compositions

 Examples of generated compositions:
  1. Na2 Fe8 O5
  2. Na2 Fe8 O7
  3. Na3 Fe1 O1
  4. Na3 Fe1 O2
  5. Na3 Fe1 O3
  6. Na3 Fe1 O4
  7. Na3 Fe1 O5
  8. Na3 Fe1 O6
  ... and 563 more

 This is the 'combinatorial explosion' we discussed earlier!


In [ ]:
print(" Applying SMACT chemical validity filters...")
print(" This checks for charge neutrality and electronegativity rules.\n")

from smact.screening import smact_validity

# Apply SMACT validity test
valid_compositions = []
invalid_count = 0

for comp in unique_compositions:
    if smact_validity(comp):
        valid_compositions.append(comp)
    else:
        invalid_count += 1

# Stats
total_generated = len(unique_compositions)
total_valid = len(valid_compositions)
filter_efficiency = (invalid_count / total_generated) * 100

# Report results
print(f" Filtering Results:")
print(f"   Total compositions generated:    {total_generated:>6}")
print(f"   Chemically valid compositions:    {total_valid:>6}")
print(f"   Invalid compositions removed:     {invalid_count:>6}")
print(f"   Filter efficiency:              {filter_efficiency:>6.1f}%")

print(f"\n SMACT filters eliminated {filter_efficiency:.1f}% of chemically impossible compositions!")

# Show examples
print(f"\n Examples of valid Na–Fe–O compositions:")
for i, comp in enumerate(valid_compositions[:10]):
    print(f"  {i+1:2d}. {comp}")
if len(valid_compositions) > 10:
    print(f"     ... and {len(valid_compositions)-10} more")


 Applying SMACT chemical validity filters...
 This checks for charge neutrality and electronegativity rules.

 Filtering Results:
   Total compositions generated:       571
   Chemically valid compositions:       164
   Invalid compositions removed:        407
   Filter efficiency:                71.3%

 SMACT filters eliminated 71.3% of chemically impossible compositions!

 Examples of valid Na–Fe–O compositions:
   1. Na2 Fe8 O5
   2. Na3 Fe1 O2
   3. Na3 Fe1 O3
   4. Na3 Fe1 O4
   5. Na3 Fe1 O5
   6. Na3 Fe1 O6
   7. Na6 Fe1 O4
   8. Na6 Fe1 O5
   9. Na6 Fe1 O6
  10. Na6 Fe1 O7
     ... and 154 more


In [ ]:
import os
os.environ["MP_API_KEY"] = "qFv4sQizMtDrhz94trtmr7r4eYz5KZ0F"


In [ ]:
print(" Querying Materials Project for known compounds in Na–Fe–O system...")

# Define the system you're working with
CHEMICAL_SYSTEM = ["Na", "Fe", "O"]

# Try loading API key (modify path if needed)
MP_API_KEY = os.environ.get("MP_API_KEY", None)
if MP_API_KEY is None:
    try:
        with open("mp_api_key.txt", "r") as f:
            MP_API_KEY = f.read().strip()
    except FileNotFoundError:
        print(" Materials Project API key not found.")
        print("Please set MP_API_KEY environment variable or create 'mp_api_key.txt'")
        MP_API_KEY = None

# If key is valid, continue querying
if MP_API_KEY:
    try:
        # Build list of chemical subsystems: Na, Fe, O, Na-Fe, Na-O, Fe-O, Na-Fe-O
        chemical_systems = []
        for r in range(1, len(CHEMICAL_SYSTEM) + 1):
            for combo in itertools.combinations(CHEMICAL_SYSTEM, r):
                chemical_systems.append("-".join(sorted(combo)))

        print(f"📡 Searching systems: {', '.join(chemical_systems)}")

        with MPRester(MP_API_KEY) as mpr:
            mp_entries = mpr.materials.summary.search(chemsys=chemical_systems)

        # Extract compositions
        mp_compositions = [entry.composition.reduced_composition for entry in mp_entries]

        print(f" Found {len(mp_compositions)} known compounds in Materials Project")

        # Show examples
        print(f"\n Examples of known Materials Project compounds:")
        for i, comp in enumerate(mp_compositions[:10]):
            print(f"  {i+1:2d}. {comp}")
        if len(mp_compositions) > 10:
            print(f"     ... and {len(mp_compositions)-10} more")

    except Exception as e:
        print(f" Error querying Materials Project: {e}")
        mp_compositions = []

else:
    print(" Skipping Materials Project query - no API key available")
    mp_compositions = []


 Querying Materials Project for known compounds in Na–Fe–O system...
📡 Searching systems: Na, Fe, O, Fe-Na, Na-O, Fe-O, Fe-Na-O


Retrieving SummaryDoc documents:   0%|          | 0/319 [00:00<?, ?it/s]

 Found 319 known compounds in Materials Project

 Examples of known Materials Project compounds:
   1. Na1
   2. Na1
   3. Na1
   4. Na1
   5. Na1
   6. Na1
   7. Na1
   8. Na1
   9. Na1
  10. Na1
     ... and 309 more


In [ ]:
print(" Analysing the gap between theory and experiment...\n")

# Compare SMACT predictions with experimental data
if mp_compositions:
    # Convert to string sets for easy comparison
    smact_formulas = set(str(comp) for comp in valid_compositions)
    mp_formulas = set(str(comp) for comp in mp_compositions)

    # Find overlaps and gaps
    both_predicted_and_known = smact_formulas.intersection(mp_formulas)
    predicted_but_unknown = smact_formulas.difference(mp_formulas)
    known_but_not_predicted = mp_formulas.difference(smact_formulas)

    print(f" COMPOSITIONAL SCREENING ANALYSIS:")
    print(f"{'='*55}")
    print(f"SMACT predicted compositions:          {len(smact_formulas):>6}")
    print(f"Materials Project known compounds:     {len(mp_formulas):>6}")
    print(f"{'='*55}")
    print(f"Correctly predicted (overlap):         {len(both_predicted_and_known):>6}")
    print(f"Predicted but not yet made:            {len(predicted_but_unknown):>6}")
    print(f"Known but not predicted by SMACT:      {len(known_but_not_predicted):>6}")
    print(f"{'='*55}")

    # Accuracy and coverage
    if len(mp_formulas) > 0:
        prediction_accuracy = len(both_predicted_and_known) / len(mp_formulas) * 100
        print(f"SMACT prediction accuracy:             {prediction_accuracy:>6.1f}%")
    if len(smact_formulas) > 0:
        experimental_coverage = len(both_predicted_and_known) / len(smact_formulas) * 100
        print(f"Experimental coverage of theory:       {experimental_coverage:>6.1f}%")

    print(f"\n Key Insight: {len(predicted_but_unknown)} compositions are predicted")
    print(f"   to be chemically feasible but haven't been synthesised yet!")
    print(f"   These represent opportunities for new Na–Fe–O battery materials.")

    # Show examples of new targets
    if predicted_but_unknown:
        unexplored_list = sorted(list(predicted_but_unknown))
        print(f"\n Examples of unexplored compositions:")
        for i, formula in enumerate(unexplored_list[:6]):
            print(f"  {i+1}. {formula}")
        if len(unexplored_list) > 6:
            print(f"     ... and {len(unexplored_list)-6} more targets for synthesis")

else:
    print(" No Materials Project data available for comparison")
    print("   Analysis will focus only on theoretical SMACT predictions.")


 Analysing the gap between theory and experiment...

 COMPOSITIONAL SCREENING ANALYSIS:
SMACT predicted compositions:             164
Materials Project known compounds:         83
Correctly predicted (overlap):             33
Predicted but not yet made:               131
Known but not predicted by SMACT:          50
SMACT prediction accuracy:               39.8%
Experimental coverage of theory:         20.1%

 Key Insight: 131 compositions are predicted
   to be chemically feasible but haven't been synthesised yet!
   These represent opportunities for new Na–Fe–O battery materials.

 Examples of unexplored compositions:
  1. Fe1 O4
  2. Fe1 O5
  3. Fe1 O6
  4. Fe2 O1
  5. Na1 Fe1
  6. Na1 Fe1 O1
     ... and 125 more targets for synthesis


In [ ]:
print(" Creating ternary plot to visualise Na–Fe–O chemical space...")

def composition_to_fractions(comp, element_list):
    """Convert composition to fractional coordinates for ternary plotting."""
    amounts = [comp[el.symbol] for el in element_list]
    total = sum(amounts)
    return [amt / total for amt in amounts] if total > 0 else [0, 0, 0]

# Get fractional coordinates for SMACT predictions
smact_fractions = np.array([composition_to_fractions(c, elements) for c in valid_compositions])
na_s, fe_s, o_s = smact_fractions[:, 0], smact_fractions[:, 1], smact_fractions[:, 2]

# Convert known MP compositions (if available)
if mp_compositions:
    mp_fractions = np.array([composition_to_fractions(c, elements) for c in mp_compositions])
    na_m, fe_m, o_m = mp_fractions[:, 0], mp_fractions[:, 1], mp_fractions[:, 2]
    print(f"Plotting {len(smact_fractions)} SMACT predictions and {len(mp_fractions)} validated MP compounds")
else:
    na_m, fe_m, o_m = [], [], []
    print(f"Plotting {len(smact_fractions)} SMACT predictions only")

# Create ternary plot
fig = go.Figure()

# SMACT predictions
fig.add_trace(go.Scatterternary(
    a=na_s, b=fe_s, c=o_s,
    mode="markers",
    marker=dict(
        size=5,
        color="lightblue",
        symbol="circle",
        opacity=0.5,
        line=dict(width=0.5, color="blue")
    ),
    name=f"SMACT Predictions ({len(valid_compositions)})",
    hovertemplate="<b>SMACT Valid Composition</b><br>" +
                  "Na: %{a:.3f}<br>" +
                  "Fe: %{b:.3f}<br>" +
                  "O: %{c:.3f}<extra></extra>"
))

# Known compounds (if available)
if len(na_m) > 0:
    fig.add_trace(go.Scatterternary(
        a=na_m, b=fe_m, c=o_m,
        mode="markers",
        marker=dict(
            size=12,
            color="red",
            symbol="star",
            opacity=0.9,
            line=dict(width=1.5, color="darkred")
        ),
        name=f"Known Compounds ({len(mp_compositions)})",
        hovertemplate="<b>Materials Project (Validated)</b><br>" +
                      "Na: %{a:.3f}<br>" +
                      "Fe: %{b:.3f}<br>" +
                      "O: %{c:.3f}<extra></extra>"
    ))

# Axis styling
axis_style = dict(
    title=dict(font=dict(size=16, color="black")),
    linewidth=2,
    linecolor="black",
    gridcolor="rgba(128, 128, 128, 0.4)",
    showticklabels=True,
    tickvals=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    tickfont=dict(size=12)
)

# Layout setup
fig.update_layout(
    title=dict(
        text="Na–Fe–O Chemical Space: Theory vs Experiment",
        font=dict(size=18, color="black"),
        x=0.5
    ),
    font=dict(size=12, family="Arial"),
    width=700,
    height=700,
    ternary=dict(
        bgcolor="rgba(250, 250, 250, 0.8)",
        aaxis=dict(axis_style, title="Sodium (Na)"),
        baxis=dict(axis_style, title="Iron (Fe)"),
        caxis=dict(axis_style, title="Oxygen (O)")
    ),
    showlegend=True,
    legend=dict(
        x=0.02, y=0.98,
        bgcolor="rgba(255,255,255,0.9)",
        bordercolor="gray",
        borderwidth=1
    ),
    paper_bgcolor="white"
)

fig.show()

print("✓ Ternary plot created successfully!")
print("\n What the plot shows:")
print("   • Blue circles: Compositions predicted by SMACT as chemically feasible")
print("   • Red stars: Compounds known from the Materials Project")
print("   • Empty regions: Parts of Na–Fe–O space with no known stable compounds")
print("   • Blue circles with no red stars: Unexplored targets for battery discovery")


 Creating ternary plot to visualise Na–Fe–O chemical space...
Plotting 164 SMACT predictions and 319 validated MP compounds


✓ Ternary plot created successfully!

 What the plot shows:
   • Blue circles: Compositions predicted by SMACT as chemically feasible
   • Red stars: Compounds known from the Materials Project
   • Empty regions: Parts of Na–Fe–O space with no known stable compounds
   • Blue circles with no red stars: Unexplored targets for battery discovery


In [ ]:
print(" Identifying the most promising compositions for experimental synthesis...\n")

if mp_compositions:
    # Identify unexplored compositions
    smact_formulas = set(str(comp) for comp in valid_compositions)
    mp_formulas = set(str(comp) for comp in mp_compositions)
    unexplored_formulas = smact_formulas.difference(mp_formulas)

    if unexplored_formulas:
        unexplored_compositions = [Composition(f) for f in unexplored_formulas]

        # Define a simple complexity metric
        def composition_complexity(comp):
            return comp.num_atoms + len(comp.elements)

        # Prioritise by simplicity
        prioritised_targets = sorted(unexplored_compositions, key=composition_complexity)

        print(f"TOP SYNTHESIS TARGETS (ordered by complexity):")
        print(f"{'Rank':<5} {'Formula':<15} {'Atoms':<8} {'Elements':<10} {'Complexity':<12}")
        print("-" * 55)

        for i, comp in enumerate(prioritised_targets[:10], 1):
            print(f"{i:<5} {str(comp):<15} {comp.num_atoms:<8} {len(comp.elements):<10} {composition_complexity(comp):<12}")

        if len(prioritised_targets) > 10:
            print(f"     ... and {len(prioritised_targets)-10} more potential targets")

        # Binary and ternary split
        binary_targets = [c for c in prioritised_targets if len(c.elements) == 2]
        ternary_targets = [c for c in prioritised_targets if len(c.elements) == 3]

        print(f"\nTarget Summary:")
        print(f"   Binary compositions (2 elements):  {len(binary_targets):>3}")
        print(f"   Ternary compositions (3 elements): {len(ternary_targets):>3}")
        print(f"   Total synthesis targets:           {len(prioritised_targets):>3}")

        # Examples
        if binary_targets:
            print(f"\nSimplest binary targets:")
            for i, comp in enumerate(binary_targets[:3], 1):
                print(f"   {i}. {comp}")

        if ternary_targets:
            print(f"\nSimplest ternary targets:")
            for i, comp in enumerate(ternary_targets[:3], 1):
                print(f"   {i}. {comp}")

        print(f"\n Recommendation: Start experimental synthesis with the simplest")
        print(f"   compositions first—they are often more feasible to produce.")

    else:
        print(" All SMACT-predicted compositions are already known!")
        print("This system appears to be thoroughly explored in the Materials Project.")

else:
    print(" Cannot identify synthesis targets without Materials Project data.")
    print("All SMACT-valid compositions could be considered for synthesis.")

    def composition_complexity(comp):
        return comp.num_atoms + len(comp.elements)

    simple_targets = sorted(valid_compositions, key=composition_complexity)[:10]

    print(f"\nSample SMACT-valid compositions to consider:")
    for i, comp in enumerate(simple_targets, 1):
        print(f"   {i:2d}. {comp} (complexity: {composition_complexity(comp)})")


 Identifying the most promising compositions for experimental synthesis...

TOP SYNTHESIS TARGETS (ordered by complexity):
Rank  Formula         Atoms    Elements   Complexity  
-------------------------------------------------------
1     Na1 Fe1         2.0      2          4.0         
2     Na2 Fe1         3.0      2          5.0         
3     Na1 Fe2         3.0      2          5.0         
4     Fe2 O1          3.0      2          5.0         
5     Na1 Fe3         4.0      2          6.0         
6     Na1 Fe1 O1      3.0      3          6.0         
7     Na3 Fe2         5.0      2          7.0         
8     Fe1 O4          5.0      2          7.0         
9     Na2 Fe3         5.0      2          7.0         
10    Na4 Fe1         5.0      2          7.0         
     ... and 121 more potential targets

Target Summary:
   Binary compositions (2 elements):   46
   Ternary compositions (3 elements):  85
   Total synthesis targets:           131

Simplest binary targets:
   1. N

In [3]:
import nbformat

# Load the notebook
# IMPORTANT: Replace 'your_notebook.ipynb' with the actual path to your notebook file
nb_path = 'Compositional screening_1.ipynb'
try:
    with open(nb_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    # Fix widgets metadata
    if 'widgets' in nb['metadata']:
        nb['metadata']['widgets'] = {'state': {}}

    # Save the notebook
    with open(nb_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    print("Fixed notebook metadata.")

except FileNotFoundError:
    print(f"Error: Notebook file not found at '{nb_path}'. Please provide the correct path.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: Notebook file not found at 'Compositional screening_1.ipynb'. Please provide the correct path.


In [4]:
!ls


sample_data


In [7]:
!ls https://drive.google.com/drive/folders/19OJVaHPhjPF08K82-GAIggZZjzzcsuKts/Compositional screening_1.ipynb


ls: cannot access 'https://drive.google.com/drive/folders/19OJVaHPhjPF08K82-GAIggZZjzzcsuKts/Compositional': No such file or directory
ls: cannot access 'screening_1.ipynb': No such file or directory
